In [50]:
#PPG(spi test set) save
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np

%load_ext autoreload
%autoreload 2
from pyvital2 import vitaldb

with open('filepath_test','rb') as f:
    e_list = pickle.load(f)
#e_list = pd.Series(np.unique(e_list))
print(len(e_list))
#print(e_list)
df_match = pd.read_csv('match_drug_arr_exclusion_20201110.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
#df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()]

e_row = []
for pathname in e_list:
    e_row.append(pathname.split(',')[1])#.str[1]
#print(e_row)

for i in e_row:
    i = int(i)
    #try: 
    
    #PACU_vital 내에서 파일 찾아서 읽기
    file_path = df_match.loc[i,'path']
    dir_PACU = 'PACU_vital/PACU1_'
    folders = ['1-3/','4-6/','7-9/','10-12/']

    for fd in folders:
        if os.path.isfile(dir_PACU + fd + file_path):
            #print(dir_PACU + fd + file_path)
            rec_path = dir_PACU+fd+file_path
            
    filename = 'PPG'+ str(df_match.loc[i,'Value'])+','+str(i)+','+ df_match.loc[i,'path']
    if not os.path.exists('SPI_test_set_new/'+filename):
        vfile = vitaldb.vital_recs(rec_path,['SNUADCW/PLETH'],interval=0.01).flatten()
        srate = 100
        #print(len(vfile[0]))
        if len(vfile[0])==0:
            print('NaN',str(i))
        
        else:
            Abs_time = [vfile[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile[0]))]
            print(i,'processing')
            df_rec = pd.DataFrame(Abs_time)
            df_rec['Abs_time']=Abs_time
            df_rec['pleth']=vfile[0]
            df_rec=df_rec.drop(0,axis=1)
            
            df_rec['Abs_time']=pd.to_datetime(df_rec['Abs_time'])
            end_idx_list = df_rec[(df_rec['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_rec['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()
            #print(df_match.loc[i, 'NRS_time'])
            #print(vfile[1])
            #print(end_idx_list)
            
            
            if len(end_idx_list)==0:
                print(str(i)+"no end idx")                  
            else:
                end_idx=end_idx_list[0]
                start_idx= 0
                extraction = df_rec.loc[start_idx:end_idx]

                with open('SPI_test_set_new/'+filename,'wb') as f:
                      pickle.dump(extraction, f)
         
                
    #except:
    #    print(rec_path)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
787
249 processing
2364 processing
5914 processing
4219 processing
8166 processing
1199 processing
6350 processing
6626 processing
3244 processing
677 processing
7398 processing
5497 processing
86 processing
1809 processing
7155 processing
303 processing
2189 processing
132 processing
460 processing
1700 processing
4010 processing
6073 processing
7055 processing
676 processing
3170 processing
5025 processing
3635 processing
2143 processing
1871 processing
252 processing
385 processing
7245 processing
7885 processing
6035 processing
6091 processing
7458 processing
7537 processing
3665 processing
3156 processing
2907 processing
7634 processing
5493 processing
5795 processing
1196 processing
7938 processing
3658 processing
4903 processing
7461 processing
3991 processing
7013 processing
7195 processing
3309 processing
3512 processing
5791 processing
2906 processing
4643 processing
2439 processing
6854 p

1250 processing
7319 processing
1640 processing
6353 processing
6991 processing
2436 processing
5572 processing
7499 processing
2686 processing
5237 processing
6762 processing
6162 processing
5147 processing
6383 processing
3996 processing
6967 processing
1459 processing
6262 processing
1072 processing
7157 processing
7072 processing
2011 processing
3937 processing
1777 processing
3462 processing
2304 processing
458 processing
4248 processing
4489 processing
4972 processing
8086 processing
3595 processing
1070 processing
6474 processing
36 processing
1083 processing
3900 processing
8054 processing
5361 processing
5733 processing
5746 processing
2236 processing
944 processing
7778 processing
159 processing
3248 processing
7876 processing
5483 processing
346 processing
5316 processing
7170 processing
5703 processing
2064 processing


In [44]:
ex = vitaldb.vital_recs('PACU_vital/PACU1_1-3/PACU1_2_190503_132609.vital',['SNUADCW/PLETH'],interval=0.01).flatten()
print(ex)
print(len(ex[0]))

[array([33.59375 , 33.59375 , 33.203125, ...,  3.125   ,  3.125   ,
        3.125   ])
 datetime.datetime(2019, 5, 3, 4, 26, 8, 505003)]
268387


In [55]:
import os

flist=[]
for (root, dirs, files) in os.walk('SPI_test_set_new/'):
    for file in files:
        flist.append(file)
print(len(flist))

787
